## 1. Import Required Libraries

In [16]:
# import libraries
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import yaml
import src.util as utils



## 2. Load config and Dataset

In [17]:
config = utils.load_config()

In [18]:
# read dataset
def read_raw_data(config: dict) -> pd.DataFrame:
    # Return raw dataset
    return pd.read_csv(config["dataset_path"])

In [19]:
dataset_diabetes = read_raw_data(config)

## 3. Data Validation

In [20]:
dataset_diabetes.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [21]:
dataset_diabetes.dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object

In [22]:
dataset_diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [23]:
dataset_diabetes.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


## 4. Data Defense

In [24]:
def check_data(input_data, config):
    len_input_data = len(input_data)
    
    # check data types
    assert input_data.select_dtypes("int").columns.to_list() == config["int_columns"], "an error occurs in int column(s)."
    assert input_data.select_dtypes("float").columns.to_list() == config["float_columns"], "an error occurs in float column(s)."
    
        # Check range of data
    assert input_data[config["float_columns"][0]].between(config["range_BMI"][0], config["range_BMI"][1]).sum() == len_input_data, "an error occurs in BMI range."
    assert input_data[config["float_columns"][1]].between(config["range_DiabetesPedigreeFunction"][0], config["range_DiabetesPedigreeFunction"][1]).sum() == len_input_data, "an error occurs in DiabetesPedigreeFunction range."
    assert input_data[config["int_columns"][0]].between(config["range_Pregnancies"][0], config["range_Pregnancies"][1]).sum() == len_input_data, "an error occurs in Pregnancies range."
    assert input_data[config["int_columns"][1]].between(config["range_Glucose"][0], config["range_Glucose"][1]).sum() == len_input_data, "an error occurs in Glucose range."
    assert input_data[config["int_columns"][2]].between(config["range_BloodPressure"][0], config["range_BloodPressure"][1]).sum() == len_input_data, "an error occurs in BloodPressure h2 range."
    assert input_data[config["int_columns"][3]].between(config["range_SkinThickness"][0], config["range_SkinThickness"][1]).sum() == len_input_data, "an error occurs in SkinThickness range."
    assert input_data[config["int_columns"][4]].between(config["range_Insulin"][0], config["range_Insulin"][1]).sum() == len_input_data, "an error occurs in Insulin range."
    assert input_data[config["int_columns"][5]].between(config["range_Age"][0], config["range_Age"][1]).sum() == len_input_data, "an error occurs in Age range."
    assert input_data[config["int_columns"][6]].between(config["range_Outcome"][0], config["range_Outcome"][1]).sum() == len_input_data, "an error occurs in Outcome range."

In [25]:
check_data(dataset_diabetes, config)

## 5. Data Splitting

In [26]:
X = dataset_diabetes[config["predictors"]].copy()
y = dataset_diabetes[config["label"]].copy()

In [27]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [28]:
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123, stratify = y)

In [30]:
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 123, stratify = y_test)

In [31]:
utils.pickle_dump(dataset_diabetes, config["dataset_cleaned_path"])

utils.pickle_dump(X_train, config["train_set_path"][0])
utils.pickle_dump(y_train, config["train_set_path"][1])

utils.pickle_dump(X_valid, config["valid_set_path"][0])
utils.pickle_dump(y_valid, config["valid_set_path"][1])

utils.pickle_dump(X_test, config["test_set_path"][0])
utils.pickle_dump(y_test, config["test_set_path"][1])